In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings("ignore")

import joblib

In [2]:
data = pd.read_csv('/kaggle/input/medicine-recommendation-system-dataset/Training.csv', delimiter=',')
data

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,(vertigo) Paroymsal Positional Vertigo
4916,0,1,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,Acne
4917,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Urinary tract infection
4918,0,1,0,0,0,0,1,0,0,0,...,0,0,1,1,1,1,0,0,0,Psoriasis


In [3]:
data = data.drop_duplicates()
data

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,Impetigo
403,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,Impetigo
405,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,Impetigo
406,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,Impetigo


In [4]:
imp_cols = ['muscle_pain',
            'fatigue',
            'mild_fever',
            'high_fever',
            'loss_of_appetite',
            'nausea',
            'vomiting',
            'itching',
            'yellowing_of_eyes',
            'headache',
            'chest_pain',
            'joint_pain',
            'dark_urine',
            'yellowish_skin',
            'breathlessness',
            'malaise',
            'diarrhoea',
            'family_history'
            ]

X, y = data[imp_cols], data['prognosis']
X.shape, y.shape

((304, 18), (304,))

In [5]:
params = {'random_strength': 1,
          'od_type': 'Iter',
          'learning_rate': 0.03,
          'l2_leaf_reg': 5,
          'iterations': 1000,
          'depth': 10,
          # 'border_count': 64,
          'bootstrap_type': 'Bernoulli',
          'auto_class_weights': 'SqrtBalanced'}

model = CatBoostClassifier(**params, cat_features=imp_cols, verbose=False)

In [6]:
model.fit(X, y)

In [7]:
joblib.dump(model, "Med_Recommender_Cat.pkl")

['Med_Recommender_Cat.pkl']

The above model turns out to be of 344.2MB so creating a smaller model for backup.

Since the params were searched with duplicates so using much less estimators sound reasonable.

In [9]:
new_params = {'random_strength': 1,
              'od_type': 'Iter',
              'learning_rate': 0.03,
              'l2_leaf_reg': 5,
              'iterations': 100,
              'depth': 10,
              'bootstrap_type': 'Bernoulli',
              'auto_class_weights': 'SqrtBalanced'}

In [10]:
model = CatBoostClassifier(**new_params, cat_features=imp_cols, verbose=False)

model.fit(X, y)

In [11]:
joblib.dump(model, "Med_Recommender_Cat_final.pkl")

['Med_Recommender_Cat_final.pkl']